In [1]:
import numpy as np
import pandas as pd

In [16]:
train_l = pd.read_csv("final_predictions_4_l.csv")
train_m = pd.read_csv("final_predictions_4_m.csv")
train_h = pd.read_csv("final_predictions_4_h.csv")
df4_17 = pd.read_csv("17_dataframe_4.csv")

In [22]:
print(train_l.shape)
print(train_m.shape)
print(train_h.shape)


(31487, 2)
(31487, 2)
(31487, 2)


In [27]:
train_h.columns = ['index', 'high']
train_m.columns = ['index', 'medium']
train_l.columns = ['index', 'low']

In [28]:
train_h.head()

,index,high
0,0,0.990836
1,1,0.990357
2,2,0.988665
3,3,0.986762
4,4,0.982742


In [33]:
train_m.head()

,index,medium
0,0,0.981471
1,1,0.973956
2,2,0.981067
3,3,0.982641
4,4,0.985286


In [34]:
train_l.head()

,index,low
0,0,0.005760
1,1,0.010163
2,2,0.009022
3,3,0.009709
4,4,0.011147


In [29]:
print(df4_17.shape)

(31487, 17)


In [30]:
df4_17.tail()

,Timesteps,ChassisId_encoded,gen,risk_level,v_category,v__0,v__1,v__2,v__3,v__4,v__5,v__6,v__7,v__8,v__9,v__10,v__11
31482,27.0,91336.0,gen1,Low,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31483,27.0,91353.0,gen1,Low,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31484,3.0,102478.0,gen1,Medium,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
31485,8.0,102478.0,gen1,High,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
31486,13.0,102478.0,gen1,High,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [31]:
df4_17 = pd.concat([df4_17, train_l[['low']], train_m[['medium']], train_h[['high']]], axis=1)

In [32]:
df4_17.head()

,Timesteps,ChassisId_encoded,gen,risk_level,v_category,v__0,v__1,v__2,v__3,v__4,v__5,v__6,v__7,v__8,v__9,v__10,v__11,low,medium,high
0,2.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.005760,0.981471,0.990836
1,7.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.010163,0.973956,0.990357
2,12.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.009022,0.981067,0.988665
3,17.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.009709,0.982641,0.986762
4,22.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.011147,0.985286,0.982742


# ***lstm***

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical


In [39]:
# Assume df is your DataFrame
# Drop unnecessary columns
columns_to_drop = ['ChassisId_encoded', 'gen', 'v_category']

# Drop columns only if they exist in the DataFrame
df4_17 = df4_17.drop(columns=[col for col in columns_to_drop if col in df4_17.columns])

# Separate features (X) and target (y)
X_train = df4_17.drop(columns=['risk_level'])
y_train = df4_17['risk_level']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_categorical = to_categorical(y_encoded)



In [41]:
print(y_categorical)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [43]:
X_train.shape

(31487, 16)

In [53]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [54]:
import random
import tensorflow as tf
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

# Build the LSTM model
X_train_new, y_train_new = X_train_reshaped, y_categorical

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_new.shape[1], X_train_new.shape[2])))
model.add(Dense(25, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_new, y_train_new, epochs=20, batch_size=32, verbose=2)

Epoch 1/20
984/984 - 5s - loss: 0.1468 - accuracy: 0.9587 - 5s/epoch - 6ms/step
Epoch 2/20
984/984 - 2s - loss: 0.0734 - accuracy: 0.9786 - 2s/epoch - 2ms/step
Epoch 3/20
984/984 - 2s - loss: 0.0680 - accuracy: 0.9807 - 2s/epoch - 2ms/step
Epoch 4/20
984/984 - 2s - loss: 0.0641 - accuracy: 0.9818 - 2s/epoch - 2ms/step
Epoch 5/20
984/984 - 3s - loss: 0.0628 - accuracy: 0.9821 - 3s/epoch - 4ms/step
Epoch 6/20
984/984 - 3s - loss: 0.0608 - accuracy: 0.9825 - 3s/epoch - 3ms/step
Epoch 7/20
984/984 - 2s - loss: 0.0590 - accuracy: 0.9828 - 2s/epoch - 2ms/step
Epoch 8/20
984/984 - 2s - loss: 0.0579 - accuracy: 0.9825 - 2s/epoch - 2ms/step
Epoch 9/20
984/984 - 2s - loss: 0.0571 - accuracy: 0.9833 - 2s/epoch - 2ms/step
Epoch 10/20
984/984 - 3s - loss: 0.0552 - accuracy: 0.9836 - 3s/epoch - 3ms/step
Epoch 11/20
984/984 - 3s - loss: 0.0537 - accuracy: 0.9842 - 3s/epoch - 4ms/step
Epoch 12/20
984/984 - 2s - loss: 0.0521 - accuracy: 0.9838 - 2s/epoch - 2ms/step
Epoch 13/20
984/984 - 2s - loss: 0.05

In [56]:
predictions = model.predict(X_train_reshaped)

# If predictions are probabilities and you want to convert them to class labels
predicted_labels = np.argmax(predictions, axis=1)

predicted_labels_encoded = label_encoder.inverse_transform(predicted_labels)


984/984 [==============================] - 4s 4ms/step


In [57]:
print(predicted_labels_encoded)

['Low' 'Low' 'Low' ... 'Medium' 'High' 'High']


In [59]:
y_train.shape

(31487,)

In [62]:
df_results = pd.DataFrame({'True': y_train, 'Pred': predicted_labels_encoded})

# Display the DataFrame
df_results.head()

,True,Pred
0,Low,Low
1,Low,Low
2,Low,Low
3,Low,Low
4,Low,Low


In [63]:
from sklearn.metrics import precision_recall_fscore_support
def macro_avg_f1(y_true, y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred)
    return round(sum(f1) / len(f1), 2)

In [64]:
score = macro_avg_f1(df_results['True'], df_results['Pred'])
print('\nScores Details:\n', score)


Scores Details:
 0.92


# **testing on train merged 1**

In [65]:
train1_l = pd.read_csv("final_predictions_l.csv")
train1_m = pd.read_csv("final_predictions_m.csv")
train1_h = pd.read_csv("final_predictions_h.csv")
df1_17 = pd.read_csv("17_dataframe_1.csv")

train1_h.columns = ['index', 'high']
train1_m.columns = ['index', 'medium']
train1_l.columns = ['index', 'low']

df1_17 = pd.concat([df1_17, train1_l[['low']], train1_m[['medium']], train1_h[['high']]], axis=1)

In [66]:
df1_17.head()

,Timesteps,ChassisId_encoded,gen,risk_level,v_category,v__0,v__1,v__2,v__3,v__4,v__5,v__6,v__7,v__8,v__9,v__10,v__11,low,medium,high
0,0.0,4953.0,gen1,High,9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044056,0.978240,0.024716
1,4.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.007713,0.977860,0.990692
2,9.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.012460,0.972358,0.989939
3,14.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.008899,0.982029,0.988074
4,19.0,4955.0,gen1,Low,33,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.010546,0.982432,0.984632


In [67]:
columns_to_drop = ['ChassisId_encoded', 'gen', 'v_category']

# Drop columns only if they exist in the DataFrame
df1_17 = df1_17.drop(columns=[col for col in columns_to_drop if col in df1_17.columns])

# Separate features (X) and target (y)
X1_test = df1_17.drop(columns=['risk_level'])
y1_test = df1_17['risk_level']

X1_test_reshaped = np.reshape(X1_test, (X1_test.shape[0], 1, X1_test.shape[1]))

predictions_1 = model.predict(X1_test_reshaped)

# If predictions are probabilities and you want to convert them to class labels
predicted_labels_1 = np.argmax(predictions_1, axis=1)

predicted_labels_encoded_1 = label_encoder.inverse_transform(predicted_labels_1)

df_results_1 = pd.DataFrame({'True': y1_test, 'Pred': predicted_labels_encoded_1})

score_1 = macro_avg_f1(df_results_1['True'], df_results_1['Pred'])
print('\nScores_1 Details:\n', score_1)

984/984 [==============================] - 4s 3ms/step

Scores_1 Details:
 0.86


# **testing on train merged 2**

In [68]:
train2_l = pd.read_csv("final_predictions_2_l.csv")
train2_m = pd.read_csv("final_predictions_2_m.csv")
train2_h = pd.read_csv("final_predictions_2_h.csv")
df2_17 = pd.read_csv("17_dataframe_2.csv")

train2_h.columns = ['index', 'high']
train2_m.columns = ['index', 'medium']
train2_l.columns = ['index', 'low']

df2_17 = pd.concat([df2_17, train2_l[['low']], train2_m[['medium']], train2_h[['high']]], axis=1)

In [69]:
columns_to_drop = ['ChassisId_encoded', 'gen', 'v_category']

# Drop columns only if they exist in the DataFrame
df2_17 = df2_17.drop(columns=[col for col in columns_to_drop if col in df2_17.columns])

# Separate features (X) and target (y)
X2_test = df2_17.drop(columns=['risk_level'])
y2_test = df2_17['risk_level']

X2_test_reshaped = np.reshape(X2_test, (X2_test.shape[0], 1, X2_test.shape[1]))

predictions_2 = model.predict(X2_test_reshaped)

# If predictions are probabilities and you want to convert them to class labels
predicted_labels_2 = np.argmax(predictions_2, axis=1)

predicted_labels_encoded_2 = label_encoder.inverse_transform(predicted_labels_2)

df_results_2 = pd.DataFrame({'True': y2_test, 'Pred': predicted_labels_encoded_2})

score_2 = macro_avg_f1(df_results_2['True'], df_results_2['Pred'])
print('\nScores_2 Details:\n', score_2)

984/984 [==============================] - 4s 4ms/step

Scores_2 Details:
 0.85


# **predicton on testing**

In [70]:
test_l = pd.read_csv("final_predictions_l.csv")
test_m = pd.read_csv("final_predictions_m.csv")
test_h = pd.read_csv("final_predictions_h.csv")
df_16 = pd.read_csv("16_dataframe.csv")

test_h.columns = ['index', 'high']
test_m.columns = ['index', 'medium']
test_l.columns = ['index', 'low']

df_16 = pd.concat([df_16, test_l[['low']], test_m[['medium']], test_h[['high']]], axis=1)

In [71]:
df_16.head()

,Timesteps,ChassisId_encoded,gen,v_category,v__0,v__1,v__2,v__3,v__4,v__5,v__6,v__7,v__8,v__9,v__10,v__11,low,medium,high
0,0.0,181.0,gen2,196,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.008943,0.975542,0.987769
1,1.0,181.0,gen2,196,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.005986,0.968864,0.992211
2,2.0,181.0,gen2,196,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.006854,0.973937,0.990114
3,3.0,181.0,gen2,196,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.007278,0.975761,0.988618
4,4.0,181.0,gen2,196,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.007198,0.976039,0.988235


In [72]:
import os
import zipfile

columns_to_drop_test = ['ChassisId_encoded', 'gen', 'v_category']

# Drop columns only if they exist in the DataFrame
df_16 = df_16.drop(columns=[col for col in columns_to_drop_test if col in df_16.columns])

# Separate features (X) and target (y)
X_test = df_16

X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

predictions = model.predict(X_test_reshaped)

# If predictions are probabilities and you want to convert them to class labels
predicted_labels = np.argmax(predictions, axis=1)

predicted_labels_encoded = label_encoder.inverse_transform(predicted_labels)

df_pred = pd.DataFrame(data = predicted_labels_encoded, columns=['pred'])

print(df_pred['pred'].value_counts())

df_pred.to_csv('prediction.csv', index=False)

def compress_file(input_file, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(input_file, os.path.basename(input_file))
input_file = 'prediction.csv'  # Input file to compress
output_zip = 'prediction.csv.zip'  # Output ZIP archive
compress_file(input_file, output_zip)

1050/1050 [==============================] - 4s 3ms/step
pred
Low       29041
High       3130
Medium     1419
Name: count, dtype: int64
